# Preprocesamiento de las Imágenes de Entrenamiento

In [3]:
#Instalamos las librerias Pillow (Manipulación de Imágenes) y Torchvision (Visión Computacional)
!pip install pillow torchvision opencv-python

   ---------------------------------------- 0.0/2.7 MB ? eta -:--:--
    --------------------------------------- 0.1/2.7 MB 1.7 MB/s eta 0:00:02
   --- ------------------------------------ 0.2/2.7 MB 2.8 MB/s eta 0:00:01
   ----- ---------------------------------- 0.4/2.7 MB 2.9 MB/s eta 0:00:01
   ----------- ---------------------------- 0.8/2.7 MB 4.6 MB/s eta 0:00:01
   -------------------- ------------------- 1.4/2.7 MB 6.2 MB/s eta 0:00:01
   ----------------------- ---------------- 1.6/2.7 MB 6.7 MB/s eta 0:00:01
   ------------------------------ --------- 2.1/2.7 MB 6.6 MB/s eta 0:00:01
   ---------------------------------------  2.7/2.7 MB 7.4 MB/s eta 0:00:01
   ---------------------------------------- 2.7/2.7 MB 7.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ------------ --------------------------- 0.5/1.7 MB 11.3 MB/s eta 0:00:01
   ---------------------- ----------------- 1.0/1.7 MB 12.1 MB/s eta 0:00:01
   ---------------------


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
#Importamos las librerias necesarias
from PIL import Image
import numpy as np
import os
from torchvision import transforms

In [1]:
#Creamos dataframes de los csv que vamos a utilizar
import pandas as pd

img_df = pd.read_csv("imagenes.csv")
annot_df = pd.read_csv("anotaciones.csv")

In [9]:
#Definimos el pipeline con las transformaciones que se van a realizar
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize((32, 128)),  # height x width
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

In [10]:
#Creamos directorio donde vamos a guardar las imagenes
os.mkdir("Anotaciones")

In [13]:
import ast

#Iteramos en cada renglon de annot_df para identificar los bordes de cada string y aplicamos las transformaciones
for idx, row in annot_df.iterrows():
    image_id = row['image_id'] + ".jpg"
    string = row['utf8_string']

    bbox = ast.literal_eval(row['bbox'])
    x, y, w, h = bbox

    img_path = os.path.join("images", image_id)
    if not os.path.exists(img_path):
        continue

    with Image.open(img_path).convert("RGB") as img:
        cropped = img.crop((x, y, x + w, y + h))
        cropped = transform(cropped)
        
        save_path = os.path.join("Anotaciones", f"{row['image_id']}_{idx}.jpg")
        transforms.ToPILImage()(cropped).save(save_path)

In [7]:
#Creamos un csv con el nombre de cada archivo y su string correspondiente para agilizar el entrenamiento
import pandas as pd
import os

labels = []

for idx, row in annot_df.iterrows():
    #Creamos la direccion para cada imagen del csv
    fname = f"{row['image_id']}_{idx}.jpg"
    fpath = os.path.join("Anotaciones", fname)

    #Agregamos tanto la direccion de la imagen y el texto correspondiente
    #Validacion por si no existe la imagen
    if os.path.exists(fpath):
        string = row['utf8_string']
        labels.append((fname, string))

#Guardamos las direcciones y texto en un DataFrame y lo exportamos a un csv
label_df = pd.DataFrame(labels, columns=["Direccion", "Texto"])
label_df.to_csv("ImagenTexto.csv", index=False)